In [1]:
import math

def euclidean_distance(vec1, vec2):
    sum = 0
    for i in range(0, len(vec1)):
        sum += math.pow(vec1[i]-vec2[i],2)
    return math.sqrt(sum)

In [2]:
def manhattan_distance(vec1, vec2):
    sum = 0
    for i in range(0, len(vec1)):
        sum += math.fabs(vec1[i] - vec2[i])
    return sum

In [17]:
#testing git

In [18]:
vec1 = [1,2,3]
vec2 = [10,2,-1]
accuracy_generalization_error(vec1, vec2)

(0.28690229202651557, 0.7130977079734844)